# Load packages, model, and parameters

In [4]:
# Load packages
import reactionmodel.parser
from reactionmodel.model import Model, eval_expression
from reactionmodel.hook import HookAwareModel
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
# Load the model

# reactionmodel.parser.load takes a path to a yaml file, and loads all the parameters / compartments / arrows / whatever specified in that file
# it returns everything it finds as a ParseResults object
#parsed = reactionmodel.parser.load('./abr.yaml')
parsed = reactionmodel.parser.load('./hook_abr.yaml', model_class=HookAwareModel)
# we access specifically the model that was parsed
# (since the file only has a model, everything else was blank, but in principle the ParseResults could have also included things like the span of time over which to simulate)
m = parsed.model
# The model is the set of all the compartments (species) and arrows (reactions).
m

In [ ]:
# Using the same method, we parse the parameters file.
parameters = reactionmodel.parser.load('./basic_parameters.yaml').parameters
# The parameters object is a mapping between names, like 'p_low_a' and numbers. Names are explained in the Google Drive.
# Values were chosen to get (1) simple (2) get the basic dynamics approximately right
parameters

In [ ]:
# Here we sift reactions to clear away "irrelevant" and "nonoperative" reactions
# Irrelevant: the parameters were such that the rate of this process is 0 no matter what.
# Nonoperative: all the reactions were specified in a programmatic way. I was careful,
# but sometimes I couldn't avoid making reactions that don't do anything like X + Y => Y + X

good = []
zero_propensity = []
noop = []
for r in m.all_reactions:
    if eval_expression(r.k, parameters) == 0:
        zero_propensity.append(r)
        continue
    elif r.reactants == r.products:
        noop.append(r)
    else:
        good.append(r)

In [ ]:
len(good), len(zero_propensity), len(noop)

In [ ]:
# Make a new model with the sifted components
reduced_model = Model(m.species, good)
reduced_model

# Simulation

In [ ]:
# Given a set of parameters, the Model object can produce the equations of motion of the system
# Here, we get the time derivative of the state: dydt(t, y) => y dot
# We set sparse=True so that it will use sparse matrices, which will be faster when we have so many reactions
# random text

dydt = reduced_model.get_dydt(parameters=parameters, sparse=True)

In [ ]:
# Creating an initial state by giving a mapping of species name => initial population
initial_pop = 500
initial = reduced_model.make_initial_condition({'<M_M_M>': initial_pop*0.8, '<M_DS_DS>': initial_pop*0.18, '<M_DR_DR>': initial_pop*0.02})

# The initial state is a vector with the following shape:
initial.shape

Below, we define a helper function `sum_and_relabel` that takes state $y(t)$ and groups together related species into a few categories for a better understanding of the state:

- If DS in the high abundance compartment, "susceptible infection"
- Otherwise, if DR in the high abundance compartment, "resistant infection"
- Otherwise, if colonized, report colonization state (DS, DR, or dual)
- Else, "uninfected uncolonized"

In [9]:
i_to_partition = {}
for i,label in enumerate(reduced_model.legend()):
    if '<DS' in label:
        partition = 'susceptible infection'
    elif '<DR' in label:
        partition = 'resistant infection'
    elif '_DS' in label and '_DR' in label:
        partition = 'dual colonization'
    elif '_DS' in label:
        partition = 'DS colonization'
    elif '_DR' in label:
        partition = 'DR colonization'
    else:
        partition = 'uninfected uncolonized'
    i_to_partition[i] = partition

def sum_and_relabel(m, state):
    partition_to_amount = {}
    for i,x in enumerate(state):
        try:
            partition_to_amount[i_to_partition[i]] += x
        except KeyError:
            partition_to_amount[i_to_partition[i]] = x
    
    return partition_to_amount

**Forwards simulation happens here:**

What we mean by simulation is realizing the time evolution of the system. Now that we `dydt` and the initial state, `initial`, we want to solve the initial value problem (how does the system evolve based on the derivative and the initial state). The package `scipy` has an integrator for this task. So below we accomplish a deterministic simulation:

In [10]:
from scipy.integrate import solve_ivp

# [0, 10.0] is the time span over which to simulate
result = solve_ivp(dydt, [0, 50.0], initial)

In [ ]:
result

The output of simulation is a `Result` object with the following relevant fields:

- `t`: the history of each time where an evaluation was made during the integration (when simulating, use the argument `t_eval=[list, of, times]` to request specific points to be evaluated)
- `y`: the state (the population of each species) evaluated at each time in the `t`. `y` is therefore a 2-D array. The first index is species, and the second index is the point in time.

In [ ]:
result.y.shape

In [13]:
# Use the sum_and_relabel tool on point in the history
new_y = []
for y in result.y.T:
    new_y.append(sum_and_relabel(reduced_model, y))

In [14]:
# Make a dataframe of the results
# A Pandas DataFrame is designed to match R dataframes, so hopefully it will be somewhat familiar 
df = pd.DataFrame(new_y)

In [ ]:
df.plot()

# Structural neutrality test

To test if the system is structurally neutral, we manipulate the parameters so that the S and R strains are identical. Then if we substitute one for the other at equilibrium, the system should not be perturbed.

In [ ]:
equal_parsed = reactionmodel.parser.load('./abr.yaml')
equal_m = equal_parsed.model

equal_parameters = reactionmodel.parser.load('./equal_strains.yaml').parameters


good = []
zero_propensity = []
noop = []
for r in m.all_reactions:
    if eval_expression(r.k, equal_parameters) == 0:
        zero_propensity.append(r)
        continue
    elif r.reactants == r.products:
        noop.append(r)
    else:
        good.append(r)

equal_m = Model(equal_m.species, good)

In [ ]:
equal_dydt = equal_m.get_dydt(parameters=equal_parameters, sparse=True)

In [65]:
from scipy.integrate import solve_ivp
equal_result = solve_ivp(equal_dydt, [0, 50.0], initial)

In [66]:
def plot(model, result, ax=None):
    ys = []
    for y in result.y.T:
        ys.append(sum_and_relabel(model, y))

    df = pd.DataFrame(ys)
    df.plot(ax=ax)

In [ ]:
plot(equal_m, equal_result)

In [68]:
initial_mostly_DS = reduced_model.make_initial_condition({'<M_M_M>': initial_pop*0.8, '<M_DS_DS>': initial_pop*0.18, '<M_DR_DR>': initial_pop*0.02})
initial_mostly_DR = reduced_model.make_initial_condition({'<M_M_M>': initial_pop*0.8, '<M_DS_DS>': initial_pop*0.02, '<M_DR_DR>': initial_pop*0.18})

initial_only_DS = reduced_model.make_initial_condition({'<M_M_M>': initial_pop*0.8, '<M_DS_DS>': initial_pop*0.20, '<M_DR_DR>': 0})
initial_only_DR = reduced_model.make_initial_condition({'<M_M_M>': initial_pop*0.8, '<M_DS_DS>': 0, '<M_DR_DR>': initial_pop*0.20})

In [69]:
mostly_DS_result = solve_ivp(equal_dydt, [0, 50.0], initial_mostly_DS)

In [ ]:
plot(equal_m, mostly_DS_result)

In [71]:
moderate = [r for r in equal_m.all_reactions if 'moderate' in r.description ]

In [72]:
from reactionmodel.model import eval_expression

for r in moderate:
    print(r)
    print(eval_expression(r.k, equal_parameters))
    break

In [73]:
mostly_DR_result = solve_ivp(equal_dydt, [0, 50.0], initial_mostly_DR)

In [ ]:
plot(equal_m, mostly_DR_result)

In [75]:
only_DS_result = solve_ivp(equal_dydt, [0, 50.0], initial_only_DS)

In [76]:
only_DR_result = solve_ivp(equal_dydt, [0, 50.0], initial_only_DR)

In [ ]:
plot(equal_m, only_DS_result)

In [ ]:
plot(equal_m, only_DR_result)

# Cutting out treatment

In [ ]:
no_treatment_parameters = reactionmodel.parser.load('./equal_no_treatment.yaml').parameters

good = []
zero_propensity = []
noop = []
for r in m.all_reactions:
    if eval_expression(r.k, no_treatment_parameters) == 0:
        zero_propensity.append(r)
        continue
    elif r.reactants == r.products:
        noop.append(r)
    else:
        good.append(r)

In [ ]:
equal_dydt_no_treatment = equal_m.get_dydt(parameters=no_treatment_parameters, sparse=True)

In [81]:
mostly_DS_result = solve_ivp(equal_dydt_no_treatment, [0, 20.0], initial_mostly_DS)

In [ ]:
plot(equal_m, mostly_DS_result)

In [83]:
mostly_DR_result = solve_ivp(equal_dydt_no_treatment, [0, 20.0], initial_mostly_DR)

In [ ]:
plot(equal_m, mostly_DR_result)

In [ ]:
no_treatment_parameters['pAMX']

In [86]:
from reactionmodel.model import eval_expression

In [87]:
def _eval_matrix(matrix_name, parameters):
    matrix = parameters[matrix_name]
    evaluated = np.zeros_like(matrix)
    it = np.nditer(matrix, flags=['refs_ok', 'multi_index'])
    for exp in it:
        evaluated[it.multi_index] = eval_expression(str(exp), parameters)
    return evaluated

In [ ]:
_eval_matrix('pAMX', no_treatment_parameters).sum()

In [ ]:
print(no_treatment_parameters['pAMX'])

In [90]:
# inner 2x2 = p_col * p_low_d
# outer edge = (1-p_col) + p_col * (1-p_low_d) (f_S + f_R)